In [71]:
pwd

'C:\\Users\\SMART-12\\Desktop\\kopofintech\\dataImport\\src\\preparation'

In [52]:
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from common import cFunction as cf
import numpy as np
import wget

In [60]:
# get dataList from filesystem to load and write
#dataList = pd.read_excel("../../data/inbound/dataList.xlsx")

# get dataList from spreadsheet to load and write
dataList = pd.read_csv("https://docs.google.com/spreadsheets/d/1VngqG-m7G8k1587c21MZoheR1Fz3amp1mJtiBvA1Jb0/export?format=csv&gid=0")
print("### The total number of target data is " + str(len(dataList)))

# Filtering -> get dataList only defined url
dataList = dataList[dataList['제공항목(데이터셋)']== '적격대출금리정보']
dataList

### The total number of target data is 417


,번호,접속 사이트,제공항목(데이터셋),폴더명,작업자,시작일,종료일,검토자,진행상태,비고,...,파라미터,제공방식.1,데이터타입,참고문서,참고문서타입,주기,비고_파라미터설명,완료여부,API PGM ID,파일명
379,380,공공데이터포털,적격대출금리정보,379_적격대출금리정보,박재원,NaN,NaN,NaN,NaN,NaN,...,NaN,api,xml,https://www.data.go.kr/comm/file/download.do?a...,doc,NaN,NaN,NaN,TB_OD10379.py,TB_OD10379_yyyymm.csv


In [72]:
print("### The total number of filtered data is " + str(len(dataList)))

###################################################
# Filtering -> for your own object
#dataList = dataList[   dataList['번호'] == "352" ]
###################################################

# create folder to save result
outPath = "../../data/outbound/"
folderList = dataList["폴더명"].tolist()

### The total number of filtered data is 1


In [73]:
for i in folderList:
    cf.createFolder(outPath+i)

In [63]:
#dataList.fillna : None 값 찾아서 ()안에 값으로 변경하기
dataList = dataList.fillna("")
dataList = dataList.reset_index(drop=True)

In [74]:
# get dataList to load and write
# for dataCount in range(0,len(dataList)):

## dataCount = 인덱스 값
dataCount =0
inputUrl = dataList.loc[dataCount, "사이트"]
inputKey = dataList.loc[dataCount, "서비스키"]
inputParameter = dataList.loc[dataCount, "파라미터"]
inputFolder = dataList.loc[dataCount, "폴더명"]
inputFile = dataList.loc[dataCount, "서비스명"]
inputDataType = dataList.loc[dataCount, "데이터타입"]
inputRefUrl = dataList.loc[dataCount, "참고문서"]
inputRefType = dataList.loc[dataCount, "참고문서타입"]
print(inputUrl)    
url = cf.makeURL(inputUrl,inputKey,inputParameter)
print("fullUrl is " + url)

http://api.hf.go.kr:8090/service/rest/conformloanrat/getConformingLoanRat?
fullUrl is http://api.hf.go.kr:8090/service/rest/conformloanrat/getConformingLoanRat?serviceKey=Z5J2q693N6mpcWIY9wSZqKgJXsAcan78Wc8dorriZfblBHoCZSqawyAVOgvykleO26uY39699xPbDd4ENw6s1w%3D%3D


In [75]:
    newDF = pd.DataFrame()
    if (inputDataType == "xml"):
        newDF = cf.xmlProcess(url)
    elif(inputDataType == "json"):
        newDF = cf.jsonProcess(url)
    elif(inputDataType == "csv"):
        newDF = cf.csvProcess(url)

    fullOutPath = outPath+inputFolder+"/"+inputFolder+inputFile+".csv"
    print(fullOutPath)

     applyDy callCenter interest_10y interest_15y interest_20y interest_30y  \
0   20190405  1566-2566         3.02         3.02         3.02         3.02   
1   20190405  1599-8000         3.25         3.25         3.25         3.25   
2   20190405  1599-0079         3.34         3.34         3.34         3.34   
3   20190405  1588-8585         3.40         3.50         3.60         3.65   
4   20190405  1588-1515         3.42         3.47         3.52         3.57   
5   20190405  1661-3000         3.47         3.52         3.57         3.62   
6   20190405  1588-6200         3.50         3.60         3.65         3.70   
7   20190405  1588-7000         3.61         3.66         3.75         3.75   
8   20190405  1588-5000         3.62         3.62         3.62         3.62   
9   20190405  1599-1111         3.79         3.84         3.89         3.89   
10  20190405  1588-2288            -            -            -         3.75   
11  20190405        기본형         None         None   

In [76]:
#csv로 저장
    try:
        newDF.to_csv(fullOutPath, index=False, encoding="ms949")
    except Exception as x:
        print(x)
##참고문서
    fullOutRefPath = outPath + inputFolder + "/" + inputFolder + inputFile + "."+inputRefType
##참고문서 저장
    try:
        wget.download(inputRefUrl, fullOutRefPath)
    except Exception as e:
        print(inputFolder+"참고문서 Error")
        print(e)
        pass

100% [....................] 1205760 / 1205760